In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

# import utils
from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial
from ImageD11.peakselect import select_ring_peaks_by_intensity

In [ ]:
# desination of H5 files

dset_path = '/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/James/20240724/FeAu_0p5_tR/FeAu_0p5_tR_ff1/FeAu_0p5_tR_ff1_dataset.h5'

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

sample = ds.sample
dataset = ds.dset

print(ds)
print(ds.shape)

In [ ]:
# load 3d columnfile from disk

cf_3d = ds.get_cf_3d_from_disk()

cf_3d.parameters.loadparameters(ds.parfile)
cf_3d.updateGeometry()

cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
cf_3d.writefile(cf_3d_path)

In [ ]:
# now we can define a unit cell from our parameters

ucell = ImageD11.unitcell.unitcell_from_parameters(cf_3d.parameters)
ucell.makerings(cf_3d.ds.max())

In [ ]:
# plot the 3D peaks (fewer of them) as a cake (two-theta vs eta)
# if the parameters in the par file are good, these should look like straight lines

fig, ax = plt.subplots()

ax.scatter(cf_3d.ds, cf_3d.eta, s=1)

ax.set_xlabel("D-star")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# here we are filtering our peaks (cf_3d) to select only the strongest ones for indexing purposes only!
# dsmax is being set to limit rings given to the indexer - 6-8 rings is normally good

# USER: modify the "frac" parameter below and re-run the cell until the orange dot sits nicely on the "elbow" of the blue line
# this indicates the fractional intensity cutoff we will select
# if the blue line does not look elbow-shaped in the logscale plot, try changing the "doplot" parameter (the y scale of the logscale plot) until it does


cf_strong_frac = 0.999
cf_strong_dsmax = 1.017
cf_strong_dstol = 0.025

cf_strong = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, doplot=0.65, dstol=cf_strong_dstol)
print(f"Got {cf_strong.nrows} strong peaks for indexing")

In [ ]:
# we will also export some additional strong peaks across all rings
# this will be useful for grain refinement later (using makemap)

cf_strong_allrings_frac = 0.999
cf_strong_allrings_dstol = 0.025

cf_strong_allrings = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_allrings_frac, dsmax=cf_3d.ds.max(), doplot=0.8, dstol=cf_strong_allrings_dstol)
print(f"Got {cf_strong_allrings.nrows} strong peaks for makemap")
cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
cf_strong_allrings.writefile(cf_strong_allrings_path)

In [ ]:
# now we can take a look at the intensities of the remaining peaks

fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)

ax.plot( ucell.ringds, [1e4,]*len(ucell.ringds), '|', ms=90, c="red")

ax.plot(cf_3d.ds, cf_3d.sum_intensity,',', label='cf_3d')
ax.plot(cf_strong.ds, cf_strong.sum_intensity,',', label='cf_strong')
ax.semilogy()

ax.set_xlabel("Dstar")
ax.set_ylabel("Intensity")
ax.legend()

plt.show()

In [ ]:
# specify our ImageD11 indexer with these peaks

indexer = ImageD11.indexing.indexer_from_colfile(cf_strong)

print(f"Indexing {cf_strong.nrows} peaks")

# USER: set a tolerance in d-space (for assigning peaks to powder rings)

indexer_ds_tol = 0.025
indexer.ds_tol = indexer_ds_tol

# change the log level so we can see what the ring assigments look like

ImageD11.indexing.loglevel = 1

# assign peaks to powder rings

indexer.assigntorings()

# change log level back again

ImageD11.indexing.loglevel = 3

In [ ]:
# let's plot the assigned peaks

fig, ax = plt.subplots()

# indexer.ra is the ring assignments

ax.scatter(cf_strong.ds, cf_strong.eta, c=indexer.ra, cmap='tab20', s=1)
ax.plot( ucell.ringds, [0,]*len(ucell.ringds), '|', ms=90, c="red")
ax.set_xlabel("d-star")
ax.set_ylabel("eta")
ax.set_xlim(cf_strong.ds.min()-0.05, cf_strong.ds.max()+0.05)

plt.show()

In [ ]:
# now we need to decide which rings to use in the grid index
# typically, 3-4 low multiplicity rings are good

rings_to_use = [0, 1, 3]

mask = np.zeros(cf_strong.nrows, dtype=bool)

for ring in rings_to_use:
    mask |= indexer.ra == ring

peaks_to_export = cf_strong.copy()
peaks_to_export.filter(mask)

In [ ]:
# now we can take a look at the intensities of the peaks we will be exporting

fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)

ax.plot( ucell.ringds, [1e4,]*len(ucell.ringds), '|', ms=90, c="red")
ax.plot(cf_3d.ds, cf_3d.sum_intensity,',', label='cf_3d')
ax.plot(peaks_to_export.ds, peaks_to_export.sum_intensity,',', label='peaks to export')
ax.semilogy()

ax.set_xlabel("Dstar")
ax.set_ylabel("Intensity")
ax.legend()

plt.show()

In [ ]:
grid_peaks_path = f'{sample}_{dataset}_3d_peaks_grid.flt'
peaks_to_export.writefile(grid_peaks_path)

In [ ]:
omegas_sorted = np.sort(ds.omega)[0]
omega_step = np.round(np.diff(omegas_sorted).mean(), 3)
omega_slop = omega_step/2

In [ ]:
# now we need to compute the number of expected peaks
# to do this, you add up the multiplicites of the rings you chose
# if you recorded a 360 degree scan, multiply the result by 2
# e.g given this output:

# info: Ring     (  h,  k,  l) Mult  total indexed to_index  ubis  peaks_per_ubi   tth
# info: Ring 3   ( -2, -2,  0)   12   2251       0     2251    93     24  16.11
# info: Ring 2   ( -1, -1, -2)   24   4899       0     4899   101     48  13.94
# info: Ring 1   ( -2,  0,  0)    6   1233       0     1233   102     12  11.37
# info: Ring 0   ( -1, -1,  0)   12   2861       0     2861   118     24  8.03

# selecting rings 0,1,3 we would get
# we would get (12+6+12)*2 = 84 peaks

In [ ]:
peaks_expected = (12+6+12)*2

# choose the fraction of the number of peaks expected - this should be around 0.9 if you had a good clean segementation
# if you suspect you are missing peaks in your data, decrease to around 0.6

frac = 0.85
minpeaks = int(np.round(peaks_expected * frac, 2))
minpeaks

In [ ]:
nproc = len(os.sched_getaffinity(os.getpid())) - 3

In [ ]:
from ImageD11.grid_index_parallel import grid_index_parallel

symmetry = "cubic"

makemap_tol_seq = [0.02, 0.015, 0.01]

gridpars = {
        'DSTOL' : 0.004,
        'OMEGAFLOAT' : omega_slop,
        'COSTOL' : np.cos(np.radians(90 - omega_slop)),
        'NPKS' : int(minpeaks),
        'TOLSEQ' : makemap_tol_seq,
        'SYMMETRY' : symmetry,
        'RING1'  : [1,0,],
        'RING2' : [0,],
        'NUL' : True,
        'FITPOS' : True,
        'tolangle' : 0.50,
        'toldist' : 100.,
        'NPROC' : nproc, # guess from cpu_count
        'NTHREAD' : 1 ,
    }
            
# grid to search
translations = [(t_x, t_y, t_z) 
    for t_x in range(-600, 601, 100)
    for t_y in range(-600, 601, 100) 
    for t_z in range(-600, 601, 100) ]
# Cylinder: 
# translations = [( x,y,z) for (x,y,z) in translations if (x*x+y*y)< 500*500 ]
#

import random
random.seed(42) # reproducible
random.shuffle(translations)

tmp_output_path = 'tmp'
map_path = 'alltmp.map'

grid_index_parallel(grid_peaks_path, ds.parfile, tmp_output_path, gridpars, translations)

In [ ]:
# re-import our refined grains from the makemap procedure

grains2 = ImageD11.grain.read_grain_file(map_path)

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d', proj_type="ortho")
xx = [grain.translation[0] for grain in grains2]
yy = [grain.translation[1] for grain in grains2]
zz = [grain.translation[2] for grain in grains2]
# col = [utils.grain_to_rgb(grain) for grain in grains2]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains2]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains2]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_aspect("equal")
plt.show()

In [ ]:
# run makemap against the selected grid peaks

symmetry = "cubic"

new_map_path = f'alltmp.map.new'
new_grid_peaks_path = f'{sample}_{dataset}_3d_peaks_grid.flt.new'

makemap_output = !makemap.py -p {ds.parfile} -u {map_path} -U {new_map_path} -f {grid_peaks_path} -s {symmetry} -t {makemap_tol_seq[-1]} --omega_slop={omega_slop} --no_sort

In [ ]:
# re-import our refined grains from the makemap procedure

grains3 = ImageD11.grain.read_grain_file(new_map_path)

In [ ]:
# remove grains with no peaks

grains3 = [grain for grain in grains3 if "no peaks" not in grain.intensity_info]

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d', proj_type="ortho")
xx = [grain.translation[0] for grain in grains3]
yy = [grain.translation[1] for grain in grains3]
zz = [grain.translation[2] for grain in grains3]
# col = [utils.grain_to_rgb(grain) for grain in grains2]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains3]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains3]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_aspect("equal")
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in grains3], bins=50)
# ax.semilogy()
plt.show()

In [ ]:
# find the spike
absolute_minpks = 20

In [ ]:
# filter out grains with fewer than absolute_minpks peaks
grains_filtered = [grain for grain in grains3 if float(grain.npks) > absolute_minpks]

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d', proj_type="ortho")
xx = [grain.translation[0] for grain in grains_filtered]
yy = [grain.translation[1] for grain in grains_filtered]
zz = [grain.translation[2] for grain in grains_filtered]
# col = [utils.grain_to_rgb(grain) for grain in grains_filtered]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains_filtered]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains_filtered]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_aspect("equal")
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
for g in grains_filtered:
    g.intensity = float(g.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))

In [ ]:
# write the filtered grains to disk

filtered_map_path = f'{sample}_{dataset}_nice_grains.map'

ImageD11.grain.write_grain_file(filtered_map_path, grains_filtered)

In [ ]:
# run makemap again against all peaks

symmetry = "cubic"

new_filtered_map_path = f'{sample}_{dataset}_nice_grains.map.new'
new_cf_3d_path = cf_3d_path + '.new'

makemap_output = !makemap.py -p {ds.parfile} -u {filtered_map_path} -U {new_filtered_map_path} -f {cf_3d_path} -s {symmetry} -t {makemap_tol_seq[-1]} --omega_slop={omega_slop} --no_sort

In [ ]:
grains_final = ImageD11.grain.read_grain_file(new_filtered_map_path)

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d', proj_type="ortho")
xx = [grain.translation[0] for grain in grains_final]
yy = [grain.translation[1] for grain in grains_final]
zz = [grain.translation[2] for grain in grains_final]
# col = [utils.grain_to_rgb(grain) for grain in grains_filtered]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains_final]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains_final]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_aspect("equal")
plt.colorbar(scatterplot)
ax.set_title("Sample 4\nGrains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in grains_final], bins=50)
# ax.semilogy()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter([float(grain.npks) for grain in grains_final], [float(g.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", "")) for g in grains_final])
ax.set_xlabel('npks')
ax.set_ylabel('sum_int')
ax.semilogy()
plt.show()

In [ ]:
len(grains_final)

In [ ]:
# import makemap output columnfile with peak assignments
cf_3d = ImageD11.columnfile.columnfile(new_cf_3d_path)

# write 3D columnfile to disk
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile, name='peaks')

In [ ]:
ds.save_grains_to_disk(grains_final)

In [ ]:
ds.save()

In [ ]:
# cleaning up

for path in [
    cf_3d_path,
    cf_strong_allrings_path,
    grid_peaks_path,
    tmp_output_path + '.flt',
    map_path,
    new_map_path,
    new_grid_peaks_path,
    filtered_map_path,
    new_filtered_map_path,
    new_cf_3d_path
]:
    if os.path.exists(path):
        os.remove(path)

In [ ]:
# change to 0 to allow all cells to be run automatically
if 1:
    raise ValueError("Hello!")

In [ ]:
# Now that we're happy with our indexing parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR": []
}

dset_prefix = "ff"

sample_list = ["FeAu_0p5_tR"]
    
samples_dict = utils.find_datasets_to_process(ds.dataroot, skips_dict, dset_prefix, sample_list)


for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        print("Importing DataSet object")
        dset_path = os.path.join(ds.analysisroot, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        if os.path.exists(ds.grainsfile):
            print(f"Found existing grains file for {dataset} in {sample}, skipping")
            continue
        
        print("Loading 3D peaks")
        cf_3d = ds.get_cf_3d_from_disk()
        cf_3d.parameters.loadparameters(ds.parfile)
        cf_3d.updateGeometry()
        cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
        cf_3d.writefile(cf_3d_path)
        
        print('Filtering peaks based on intensity')
        cf_strong = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, dstol=cf_strong_dstol)
        cf_strong_allrings = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_allrings_frac, dsmax=cf_3d.ds.max(), dstol=cf_strong_allrings_dstol)
        cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
        cf_strong_allrings.writefile(cf_strong_allrings_path)
        
        print('Filtering peaks to rings for grid index')
        indexer = ImageD11.indexing.indexer_from_colfile(cf_strong)
        indexer.ds_tol = indexer_ds_tol
        indexer.assigntorings()
        mask = np.zeros(cf_strong.nrows, dtype=bool)
        for ring in rings_to_use:
            mask |= indexer.ra == ring
        peaks_to_export = cf_strong.copy()
        peaks_to_export.filter(mask)
        grid_peaks_path = f'{sample}_{dataset}_3d_peaks_grid.flt'
        peaks_to_export.writefile(grid_peaks_path)
        
        omegas_sorted = np.sort(ds.omega)[0]
        omega_step = np.round(np.diff(omegas_sorted).mean(), 3)
        omega_slop = omega_step/2
        
        print('Launching grid index')
        grid_index_parallel(grid_peaks_path, ds.parfile, tmp_output_path, gridpars, translations)
        
        print('Running makemap against grid peaks')
        new_grid_peaks_path = f'{sample}_{dataset}_3d_peaks_grid.flt.new'
        makemap_output = !makemap.py -p {ds.parfile} -u {map_path} -U {new_map_path} -f {grid_peaks_path} -s {symmetry} -t {makemap_tol_seq[-1]} --omega_slop={omega_slop} --no_sort
        
        print('Filtering grains based on minimum peaks')
        grains3 = ImageD11.grain.read_grain_file(new_map_path)
        grains3 = [grain for grain in grains3 if "no peaks" not in grain.intensity_info]
        grains_filtered = [grain for grain in grains3 if float(grain.npks) > absolute_minpks]
        
        print('Running final makemap against all peaks')
        filtered_map_path = f'{sample}_{dataset}_nice_grains.map'
        ImageD11.grain.write_grain_file(filtered_map_path, grains_filtered)
        new_filtered_map_path = f'{sample}_{dataset}_nice_grains.map.new'
        new_cf_3d_path = cf_3d_path + '.new'
        makemap_output = !makemap.py -p {ds.parfile} -u {filtered_map_path} -U {new_filtered_map_path} -f {cf_3d_path} -s {symmetry} -t {makemap_tol_seq[-1]} --omega_slop={omega_slop} --no_sort
        grains_final = ImageD11.grain.read_grain_file(new_filtered_map_path)
        ds.save_grains_to_disk(grains_final)

        cf_3d = ImageD11.columnfile.columnfile(new_cf_3d_path)
        ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile, name='peaks')
        
        ds.save()
        
        print('Cleaning up')
        for path in [
            cf_3d_path,
            cf_strong_allrings_path,
            grid_peaks_path,
            tmp_output_path + '.flt',
            map_path,
            new_map_path,
            new_grid_peaks_path,
            filtered_map_path,
            new_filtered_map_path,
            new_cf_3d_path
        ]:
            if os.path.exists(path):
                os.remove(path)

print("Done!")